In [1]:
import pandas as pd
import glob
import os
from collections import Counter

In [232]:
#https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas

pd.set_option('mode.chained_assignment', None)

In [15]:
#data source
#https://premium.pff.com/ncaa/positions/2014/REGPO/receiving?division=fbs&position=WR

In [2]:
path = r'C:\Users\16467\Documents\OMSA\CDA_ISYE6740_Spring24\project\data' 
all_files = glob.glob(path + "/*.csv")

In [3]:
#all_files

In [4]:
#https://stackoverflow.com/questions/56949605/how-to-read-each-file-from-a-folder-and-create-seperate-data-frames-for-each-fil
d = {os.path.splitext(os.path.basename(f))[0] : pd.read_csv(f) for f in all_files} 

In [5]:
#create a list of tuples where each tuple is (name of csv,dataframe)

#https://stackoverflow.com/questions/10795973/python-dictionary-search-values-for-keys-using-regular-expression
ncaa_list = [(key, value) for key, value in d.items() if key.startswith("ncaa_")]

#add year column to each dataframe

for entry in ncaa_list:
    if 'year' not in entry[1].columns:
        name = entry[0][len(entry[0]) - 2:]
        year = "20"+name
        entry[1].insert(0, 'year', int(year))

#concat dfs in list into one df
ncaa_all = pd.DataFrame()
for i in ncaa_list:
    ncaa_all = pd.concat([ncaa_all,i[1]])

In [6]:
#do the above for NFL data

nfl_list = [(key, value) for key, value in d.items() if key.startswith("nfl_")]

for entry in nfl_list:
    if 'year' not in entry[1].columns:
        name = entry[0][len(entry[0]) - 2:]
        year = "20"+name
        entry[1].insert(0, 'year', int(year))

nfl_all = pd.DataFrame()
for i in nfl_list:
    nfl_all = pd.concat([nfl_all,i[1]])

In [7]:
nflWR = nfl_all.loc[nfl_all['position'] == 'WR']
ncaaWR = ncaa_all.loc[ncaa_all['position']  == 'WR']

In [8]:
#drop all columns that are not player and player_id
merged = pd.merge(ncaaWR, nflWR, left_on=['player','player_id'], right_on=['player','player_id'], how='inner')
columns_drop = list((Counter(list(merged.columns))-Counter(['player','player_id'])).elements())
merged.drop(columns=columns_drop, inplace=True)

#get the list of WRs who are in both the NCAA and NFL datasets
player_list = merged.drop_duplicates()

#Create separate dfs with WR players who have played in the NCCA and the NFL
ncaa = ncaaWR[ncaaWR['player_id'].isin(player_list['player_id'])]
nfl = nflWR[nflWR['player_id'].isin(player_list['player_id'])]

In [9]:
unique_ncaa = ncaa.drop_duplicates('player_id')
print(f"We have {len(unique_ncaa)} players who have played in both the NCAA between 2014-2022 and the NFL between 2015-2023")

We have 377 players who have played in both the NCAA between 2014-2022 and the NFL between 2015-2023


In [10]:
ncaa = ncaa.reset_index(drop=True)
nfl = nfl.reset_index(drop=True)

In [32]:
nfl.columns

Index(['year', 'player', 'player_id', 'position', 'team_name',
       'player_game_count', 'avg_depth_of_target', 'avoided_tackles',
       'caught_percent', 'contested_catch_rate', 'contested_receptions',
       'contested_targets', 'declined_penalties', 'drop_rate', 'drops',
       'first_downs', 'franchise_id', 'fumbles', 'grades_hands_drop',
       'grades_hands_fumble', 'grades_offense', 'grades_pass_block',
       'grades_pass_route', 'inline_rate', 'inline_snaps', 'interceptions',
       'longest', 'pass_block_rate', 'pass_blocks', 'pass_plays', 'penalties',
       'receptions', 'route_rate', 'routes', 'slot_rate', 'slot_snaps',
       'targeted_qb_rating', 'targets', 'touchdowns', 'wide_rate',
       'wide_snaps', 'yards', 'yards_after_catch',
       'yards_after_catch_per_reception', 'yards_per_reception', 'yprr'],
      dtype='object')

In [28]:
cols_keep = ['year', 'player', 'player_id','player_game_count', 'avg_depth_of_target', 'avoided_tackles','caught_percent','drops','grades_hands_drop',
       'grades_hands_fumble', 'grades_offense','grades_pass_route','route_rate', 'routes', 'slot_rate', 'slot_snaps',
       'targeted_qb_rating', 'targets', 'touchdowns', 'wide_rate','wide_snaps','routes','receptions', 'yards', 'yards_after_catch','yards_after_catch_per_reception', 'yards_per_reception', 'yprr']

In [29]:
len(cols_keep)

28

In [230]:
nfl_filter = nfl[cols_keep]
groupby_NFL = nfl_filter.groupby('player_id')

In [39]:
ncaa_filter = ncaa[cols_keep]
groupby_ncaa = ncaa_filter.groupby('player_id')

In [227]:
#Replace all Nans with column means

final_season = groupby_ncaa.tail(1)
nan_cols = list(final_season.columns[final_season.isnull().any(axis=0)])
nanDF = final_season.loc[:,nan_cols].copy(deep=False)

for i in nan_cols:     #---Applying Only on variables with NaN values
    nanDF.loc[:,i].fillna(nanDF.loc[:,i].mean(), inplace=True)
    final_season[i] = nanDF[i]
    
final_season.isnull().values.any()

False

In [233]:
final_season = final_season.reset_index(drop=True)

In [234]:
#NFL WRs who had less than 10 games played in their final college season

final_season.loc[final_season['player_game_count'] < 10]

,year,player,player_id,player_game_count,avg_depth_of_target,avoided_tackles,caught_percent,drops,grades_hands_drop,grades_hands_fumble,...,touchdowns,wide_rate,wide_snaps,routes,receptions,yards,yards_after_catch,yards_after_catch_per_reception,yards_per_reception,yprr
21,2014,DeVante Parker,9447,6,14.4,15,57.3,2,79.100000,73.2,...,5,89.8,185,201,43,855,348,8.100000,19.900000,4.25
75,2015,Leonte Carroo,10720,8,14.6,7,68.4,2,77.700000,72.9,...,10,92.5,186,194,39,808,266,6.800000,20.700000,4.16
84,2015,Johnny Holton,11354,8,18.6,4,56.3,1,72.800000,69.9,...,5,92.7,165,166,18,478,192,10.700000,26.600000,2.88
88,2015,D'haquille Williams,11366,5,13.2,0,52.2,1,66.400000,68.5,...,1,27.8,30,105,12,147,32,2.700000,12.300000,1.40
129,2016,Bobo Wilson,12067,7,9.6,3,61.2,1,80.500000,79.1,...,1,91.3,190,198,30,390,107,3.600000,13.000000,1.97
131,2016,Mack Hollins,11873,7,18.0,0,51.6,1,70.900000,69.5,...,4,91.7,144,145,16,309,52,3.300000,19.300000,2.13
133,2016,Cody Hollister,12052,9,15.3,0,62.5,3,33.200000,69.5,...,0,92.6,75,76,10,140,17,1.700000,14.000000,1.84
135,2016,Chris Thompson,12330,3,16.0,0,0.0,0,69.190426,6.9,...,0,85.7,12,10,0,0,0,5.753457,14.751862,0.00
172,2017,Richie James,28991,5,8.7,1,55.4,5,51.200000,28.2,...,3,7.5,12,150,31,290,138,4.500000,9.400000,1.93
174,2017,Chad Beebe,21528,9,10.9,0,65.1,5,48.300000,81.0,...,1,29.5,57,183,28,419,183,6.500000,15.000000,2.29


In [245]:
final_season.describe()

,year,player_id,player_game_count,avg_depth_of_target,avoided_tackles,caught_percent,drops,grades_hands_drop,grades_hands_fumble,grades_offense,...,touchdowns,wide_rate,wide_snaps,routes,receptions,yards,yards_after_catch,yards_after_catch_per_reception,yards_per_reception,yprr
count,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,...,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000,377.000000
mean,2017.745358,42792.116711,11.811671,12.319098,8.944297,64.256764,4.527851,69.190426,64.283289,75.982228,...,6.809019,63.308223,223.694960,335.859416,56.299735,814.981432,326.206897,5.753457,14.751862,2.381592
std,2.488032,31188.328138,2.234657,3.278404,6.332924,9.352927,2.698481,13.901958,15.953658,9.517860,...,4.232677,32.099065,142.746315,113.364029,26.453951,381.246907,187.749620,2.041286,3.385136,0.750247
min,2014.000000,9437.000000,3.000000,2.600000,0.000000,0.000000,0.000000,17.900000,6.900000,48.600000,...,0.000000,0.500000,1.000000,10.000000,0.000000,0.000000,0.000000,1.500000,6.800000,0.000000
25%,2016.000000,11827.000000,11.000000,10.400000,4.000000,58.900000,3.000000,60.800000,54.600000,70.000000,...,4.000000,32.900000,90.000000,269.000000,37.000000,525.000000,190.000000,4.300000,12.700000,1.910000
50%,2018.000000,42312.000000,12.000000,12.000000,8.000000,64.300000,4.000000,71.400000,72.300000,76.200000,...,6.000000,77.100000,232.000000,346.000000,55.000000,796.000000,293.000000,5.500000,14.300000,2.340000
75%,2020.000000,61398.000000,13.000000,14.500000,13.000000,70.000000,6.000000,78.800000,75.500000,83.300000,...,10.000000,88.700000,332.000000,414.000000,73.000000,1084.000000,435.000000,6.900000,16.100000,2.830000
max,2022.000000,129799.000000,15.000000,23.800000,33.000000,100.000000,19.000000,93.300000,87.800000,94.900000,...,23.000000,100.000000,575.000000,625.000000,158.000000,1856.000000,1055.000000,15.000000,34.700000,4.940000


In [242]:
thousand = nfl.loc[nfl['yards'] >= 1000]
len(thousand['player_id'].unique())

41

In [243]:
eighthunn = nfl.loc[nfl['yards'] >= 800]
len(eighthunn['player_id'].unique())

63